# PCT Evaluation

Load and test trained models, and show various evaluation metrics.

## Prep

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
!git clone --recursive --single-branch --branch pct-dataloader https://github.com/DavidClaszen/pointcloud-bench /content/pointcloud-bench
%cd /content/pointcloud-bench
%pip install -r envs/pct/requirements.txt

In [ ]:
# PAPNet partials
ZIP = '/content/drive/MyDrive/pointcloud-bench/datasets/partialmodelnet40.zip'
DEST = '/content/modelnet40'
!mkdir -p "{DEST}"
!unzip -q -n "{ZIP}" -d "{DEST}"
!find "{DEST}" -type f | wc -l

In [ ]:
# Original data used by Point-Transformers, from ShapeNet
ZIP = '/content/drive/MyDrive/pointcloud-bench/datasets/modelnet40_normal_resampled.zip'
DEST = '/content/modelnet40'
!unzip -q -n '{ZIP}' -d '{DEST}'
!find '{DEST}' -type f | wc -l

## Evaluate

Upload models, get test results, visualize and analyze.

In [ ]:
from google.colab import files
files.upload()

In [ ]:
# Can run it from the command line but... not really useful
!python test_cls.py data_path=../../../modelnet40/ checkpoint_path=../../../best_model.pth

In [ ]:
from hydra import initialize_config_dir, compose
from test_cls import get_predictions
from omegaconf import OmegaConf

config_dir = "/content/pointcloud-bench/repos/Point-Transformers/config"

with initialize_config_dir(config_dir=config_dir, version_base="1.2"):
    cfg = compose(
        config_name="cls",
        overrides=[
            "data_path=../../../modelnet40/",
            "checkpoint_path=best_model.pth",
            "use_papnet_loader=false"
        ],
    )

OmegaConf.set_struct(cfg, False)
y_true, y_pred = get_predictions(cfg)


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt

cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()